In [1]:
import setup

setup.init_django()

In [2]:
from rag import (
    db as rag_db, 
    engines as rag_engines,
    settings as rag_settings, 
    updaters as rag_updaters,
)

In [3]:
from typing import Optional, Union
from sqlalchemy import create_engine, text

In [4]:
rag_settings.init()
rag_db.init_vector_db()
# rag_updaters.update_llama_index_documents(use_saved_embeddings=True)

In [5]:
vector_index = rag_engines.get_semantic_query_index()
semantic_query_retriever = rag_engines.get_semantic_query_retriever_engine()
sql_query_engine = rag_engines.get_sql_query_engine()

In [6]:
print(rag_settings.VECTOR_DB_NAME, rag_settings.VECTOR_DB_TABLE_NAME)

vector_db blogpost


In [7]:
from llama_index.core.tools import QueryEngineTool

vector_tool = QueryEngineTool.from_defaults(
    query_engine=semantic_query_retriever,
    description=(
        f"Useful for answering semantic questions about different blog posts"
    ),
)

In [8]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over"
        " a table containing: blog posts and page views each blog post"
    ),
)

In [9]:
import logging
from typing import Any, Optional, Union

from llama_index.core.callbacks.base import CallbackManager
from llama_index.core.query_engine.sql_vector_query_engine import DEFAULT_SQL_VECTOR_SYNTHESIS_PROMPT
from llama_index.core.query_engine import SQLAutoVectorQueryEngine
from llama_index.core.indices.struct_store.sql_query import (
    BaseSQLTableQueryEngine,
    NLSQLTableQueryEngine,
)
from llama_index.core.indices.vector_store.retrievers.auto_retriever import (
    VectorIndexAutoRetriever,
)
from llama_index.core.llms.llm import LLM
from llama_index.core.prompts.base import BasePromptTemplate, PromptTemplate
from llama_index.core.prompts.mixin import PromptDictType, PromptMixinType
from llama_index.core.query_engine.retriever_query_engine import (
    RetrieverQueryEngine,
)
from llama_index.core.query_engine.sql_join_query_engine import (
    SQLAugmentQueryTransform,
    SQLJoinQueryEngine,
)
from llama_index.core.selectors.llm_selectors import LLMSingleSelector
from llama_index.core.selectors.pydantic_selectors import PydanticSingleSelector
from llama_index.core.tools.query_engine import QueryEngineTool


class MySQLAutoVectorQueryEngine(SQLAutoVectorQueryEngine):
    def __init__(
        self,
        sql_query_tool: QueryEngineTool,
        vector_query_tool: QueryEngineTool,
        selector: Optional[Union[LLMSingleSelector, PydanticSingleSelector]] = None,
        llm: Optional[LLM] = None,
        # service_context: Optional[ServiceContext] = None,
        
        sql_vector_synthesis_prompt: Optional[BasePromptTemplate] = None,
        sql_augment_query_transform: Optional[SQLAugmentQueryTransform] = None,
        use_sql_vector_synthesis: bool = True,
        callback_manager: Optional[CallbackManager] = None,
        verbose: bool = True,
    ) -> None:
        """Initialize params."""
        # validate that the query engines are of the right type
        if not isinstance(
            sql_query_tool.query_engine,
            (BaseSQLTableQueryEngine, NLSQLTableQueryEngine),
        ):
            raise ValueError(
                "sql_query_tool.query_engine must be an instance of "
                "BaseSQLTableQueryEngine or NLSQLTableQueryEngine"
            )
        if not isinstance(vector_query_tool.query_engine, RetrieverQueryEngine):
            raise ValueError(
                "vector_query_tool.query_engine must be an instance of "
                "RetrieverQueryEngine"
            )
        # if not isinstance(
        #     vector_query_tool.query_engine.retriever, VectorIndexAutoRetriever
        # ):
        #     raise ValueError(
        #         "vector_query_tool.query_engine.retriever must be an instance "
        #         "of VectorIndexAutoRetriever"
        #     )

        sql_vector_synthesis_prompt = (
            sql_vector_synthesis_prompt or DEFAULT_SQL_VECTOR_SYNTHESIS_PROMPT
        )
        SQLJoinQueryEngine.__init__( # This class also needs to be imported
            self,
            sql_query_tool,
            vector_query_tool,
            selector=selector,
            llm=llm,
            # service_context=service_context,
            sql_join_synthesis_prompt=sql_vector_synthesis_prompt,
            sql_augment_query_transform=sql_augment_query_transform,
            use_sql_join_synthesis=use_sql_vector_synthesis,
            callback_manager=callback_manager,
            verbose=verbose,
        )

In [10]:
# from llama_index.core.query_engine import SQLAutoVectorQueryEngine

query_engine = MySQLAutoVectorQueryEngine(
    sql_tool, 
    vector_tool,
)

In [11]:
response = query_engine.query(
    "What kind of Monopoly and network effects is discussed?"
)

Querying other query engine: The question 'What kind of Monopoly and network effects is discussed?' is a semantic question about the content of blog posts. Choice (2) is described as being useful for answering semantic questions about different blog posts, making it the more relevant choice.


In [12]:
response.response

'The text contrasts old-fashioned monopolies that rely on coercion with modern monopolies that rely on the network effect.\n'

In [13]:
response = query_engine.query(
    "What kind of org is discussed?"
)
response.response

Querying other query engine: Answering 'What kind of org is discussed?' is a semantic question about blog posts, making choice 2 the most relevant.


'The discussion contrasts an organization, which is structured and requires approval for changes, with an organism, which is constantly changing and adapts to its environment.\n'

In [14]:
response = query_engine.query(
    "Are are the top 5 most viewed blog posts? What keywords do their content have?"
)

Querying SQL database: The question requires identifying the top 5 most viewed blog posts and extracting keywords from their content. This involves querying a table containing blog posts and page views, and then analyzing the content of those posts. Choice 1 directly addresses translating a natural language query into a SQL query for such a table.
SQL query: SELECT
  bp.title,
  bp._content
FROM analytics_pageview AS ap
JOIN blog_blogpost AS bp
  ON ap.post_id = bp.id
GROUP BY
  bp.title,
  bp._content
ORDER BY
  COUNT(ap.post_id) DESC
LIMIT 5;
SQL response: Here are the top 5 most viewed blog posts based on the query results, along with a summary of their content to identify potential keywords:

1.  **Toward Better:** This post discusses the non-linear path to improvement, the temporary nature of struggles, and the opportunities hidden within problems.
    *   **Keywords:** improvement, progress, opportunities, problems, perseverance.

2.  **Taking it Very Seriously:** This post refle

In [15]:
from IPython.display import Markdown, display

display(Markdown(response.response))

The top 5 most viewed blog posts are: "Toward Better," "Taking it Very Seriously," "Enough Little Things," "Communicating Online (The Big Leaps)," and "You Are Your Clips."  "Toward Better" discusses improvement and opportunities. "Taking it Very Seriously" reflects on April Fool's Day and internet culture. "Enough Little Things" emphasizes small acts of kindness. "Communicating Online (The Big Leaps)" discusses online communication and avoiding spam. "You Are Your Clips" argues that our actions define our contribution.


In [16]:
response = query_engine.query(
    "What are the top 5 least viewed blog posts from today? What are the numbers of views?"
)
print(response.response)

Querying SQL database: The question requires retrieving specific data (top 5 least viewed blog posts and their view counts) from a table containing blog posts and page views. This aligns directly with the description of choice (1), which involves translating a natural language query into a SQL query to access such data.
SQL query: SELECT
  bp.title,
  COUNT(ap.post_id) AS view_count
FROM blog_blogpost AS bp
LEFT JOIN analytics_pageview AS ap
  ON bp.id = ap.post_id
WHERE
  DATE(ap.timestamp) = CURRENT_DATE
GROUP BY
  bp.title
ORDER BY
  view_count ASC
LIMIT 5;
SQL response: The top 5 least viewed blog posts from today are: "Self, community and motivation" with 550 views, "You’re surrounded" with 557 views, "Generous isn’t always the same as free" with 585 views, "On predicting the future" with 630 views, and "Monopoly and network effects" with 710 views.

Transformed query given SQL response: None

query engine response: Here are some of the topics covered:

*   How to approach each da

In [17]:
display(Markdown(response.response))

The top 5 least viewed blog posts from today are: "Self, community and motivation" with 550 views, "You’re surrounded" with 557 views, "Generous isn’t always the same as free" with 585 views, "On predicting the future" with 630 views, and "Monopoly and network effects" with 710 views.
